# CPA

Another approach to side-channel attacks is Correlation Power Attack. Main idea of this type of attack is that the **correlation** between inputs and secret key can be observed in captured power traces. Similar to DPA, attacker guesses the key and then inspects crypto outputs. Large correlation coefficient values indicate that the key guess was correct.

In other words, attacker is trying to accurately produce a power model for a device under attack.

CPA steps:
* Send random plaintext to the device 
* Measure power consumption during encryption
* Make a key guess and calculate expected values based on that guess and a device's assumed power model
* Calculate Pearson's correlation coefficient between predicted and actual powers
* Chose a byte with a greatest correlation coefficient; This byte is predicted key byte
* Repeat for all the other key bytes

In [13]:
import os
import numpy as np
import random

In [11]:
from utils.data_preparation import SCAML_Dataset

In [7]:
EXECUTE_IN_COLAB = False
ATTACK_ALGORITHM = 'tinyaes'

if EXECUTE_IN_COLAB:
    from google.colab import drive
    drive.mount('/content/drive')
    
    colab_root_path = '/content/drive/MyDrive/'
    
    training_data_path = colab_root_path + 'datasets/tinyaes/train'
    testing_data_path = colab_root_path + 'datasets/tinyaes/test'
    
    save_models_root_path = colab_root_path + 'models/'
    
    log_root_path = colab_root_path + 'logs/'
else:
    data_root_path = './data/SCA_datasets/datasets/' + ATTACK_ALGORITHM
    
    training_data_path = data_root_path + '/train'
    testing_data_path = data_root_path + '/test'
    
    save_models_root_path = './models/'
    
    log_root_path = './logs/'

In [15]:
dataset = SCAML_Dataset()
# Load attack shards
shard_array = dataset.load_shards(testing_data_path)

256it [00:00, 5044.81it/s]


In [17]:
test_shard = shard_array[random.randint(0, len(shard_array))]

In [18]:
correct_key = test_shard['keys'][:,0]
print(f"Correct key: {correct_key}")

Correct key: [125   3  25 126 249  41  85  69 106 159 127 133 139 202  50 115]


In [19]:
AES_SBOX = [
    # 0    1    2    3    4    5    6    7    8    9    a    b    c    d    e    f 
    0x63,0x7c,0x77,0x7b,0xf2,0x6b,0x6f,0xc5,0x30,0x01,0x67,0x2b,0xfe,0xd7,0xab,0x76, # 0
    0xca,0x82,0xc9,0x7d,0xfa,0x59,0x47,0xf0,0xad,0xd4,0xa2,0xaf,0x9c,0xa4,0x72,0xc0, # 1
    0xb7,0xfd,0x93,0x26,0x36,0x3f,0xf7,0xcc,0x34,0xa5,0xe5,0xf1,0x71,0xd8,0x31,0x15, # 2
    0x04,0xc7,0x23,0xc3,0x18,0x96,0x05,0x9a,0x07,0x12,0x80,0xe2,0xeb,0x27,0xb2,0x75, # 3
    0x09,0x83,0x2c,0x1a,0x1b,0x6e,0x5a,0xa0,0x52,0x3b,0xd6,0xb3,0x29,0xe3,0x2f,0x84, # 4
    0x53,0xd1,0x00,0xed,0x20,0xfc,0xb1,0x5b,0x6a,0xcb,0xbe,0x39,0x4a,0x4c,0x58,0xcf, # 5
    0xd0,0xef,0xaa,0xfb,0x43,0x4d,0x33,0x85,0x45,0xf9,0x02,0x7f,0x50,0x3c,0x9f,0xa8, # 6
    0x51,0xa3,0x40,0x8f,0x92,0x9d,0x38,0xf5,0xbc,0xb6,0xda,0x21,0x10,0xff,0xf3,0xd2, # 7
    0xcd,0x0c,0x13,0xec,0x5f,0x97,0x44,0x17,0xc4,0xa7,0x7e,0x3d,0x64,0x5d,0x19,0x73, # 8
    0x60,0x81,0x4f,0xdc,0x22,0x2a,0x90,0x88,0x46,0xee,0xb8,0x14,0xde,0x5e,0x0b,0xdb, # 9
    0xe0,0x32,0x3a,0x0a,0x49,0x06,0x24,0x5c,0xc2,0xd3,0xac,0x62,0x91,0x95,0xe4,0x79, # a
    0xe7,0xc8,0x37,0x6d,0x8d,0xd5,0x4e,0xa9,0x6c,0x56,0xf4,0xea,0x65,0x7a,0xae,0x08, # b
    0xba,0x78,0x25,0x2e,0x1c,0xa6,0xb4,0xc6,0xe8,0xdd,0x74,0x1f,0x4b,0xbd,0x8b,0x8a, # c
    0x70,0x3e,0xb5,0x66,0x48,0x03,0xf6,0x0e,0x61,0x35,0x57,0xb9,0x86,0xc1,0x1d,0x9e, # d
    0xe1,0xf8,0x98,0x11,0x69,0xd9,0x8e,0x94,0x9b,0x1e,0x87,0xe9,0xce,0x55,0x28,0xdf, # e
    0x8c,0xa1,0x89,0x0d,0xbf,0xe6,0x42,0x68,0x41,0x99,0x2d,0x0f,0xb0,0x54,0xbb,0x16  # f
]

def process_aes(input_byte, key_byte):
  sbox_input = input_byte ^ key_byte
  return AES_SBOX[sbox_input]

Iz same implementacije elektronskih komponenata sistema proizilazi cinjenica da se sa vecim brojem jedinica u binarnom zapisu podataka trosi vise struje. CPA koristi ovu cinjenicu da modeluje potrosnju. Broj jedinica nekog broja se naziva Hamingova tezina.

In [20]:
def hamming_weight(num):
  hw = bin(num).count('1')
  return hw

In [21]:
hamming_weight(0x24)

2

In [10]:
test_shard['pts'][1].shape

(256,)

In [22]:
from tqdm import trange

# Attack single byte
key_byte_index = 0

traces = test_shard['traces'][:,:,0]
traces_mean = np.mean(test_shard['traces'], axis=0)
traces_stddev = np.std(test_shard['traces'], axis=0)

traces_diff = traces - traces_mean.transpose()

cpa_outputs = 256*[0]

for key_guess in trange(0,256, position=0, leave=True):

  hamming_weights = [hamming_weight(process_aes(byte, key_guess)) for byte in test_shard['pts'][key_byte_index]]

  # Pearson's correlation coefficient:
  # 
  #
  # r = cov(X,Y) / stddev(X)*stddev(Y)
  # 
  # cov(X,Y) = E[(X-X_)*(Y-Y_)]   => sum((X-))
  # stddev(X) = sqrt((X-X_)^2)

  hamming_weights_mean = np.mean(hamming_weights)
  hamming_weights_stddev = np.std(hamming_weights)
  hamming_weights_diff = (hamming_weights - hamming_weights_mean).reshape((256,1))
  
  covariance = np.sum(hamming_weights_diff*traces_diff, axis=0)
  pearson_correlation = covariance/((hamming_weights_stddev * traces_stddev).transpose())
  
  # Point in the trace where the correlation is biggest -> where the leak is happening
  cpa_outputs[key_guess] = np.max(abs(pearson_correlation))

predicted_key_values = np.asarray(cpa_outputs)[np.argsort(cpa_outputs)][::-1]
print(f"Top 5 key byte predictions: {predicted_key_values[:5]}")

print(f"Correct key byte: {correct_key[key_byte_index]}")

100%|██████████████| 256/256 [00:26<00:00,  9.66it/s]

Top 5 key byte predictions: [215.88852459  77.84987893  76.59183673  76.37506336  76.28249744]
Correct key byte: 125


In [23]:
cpa_results_indices = np.argsort(cpa_outputs)[::-1]
cpa_results_coefficients = np.asarray(cpa_outputs)[cpa_results_indices]

print(f"Top 5 key byte predictions: {cpa_results_indices[:5]} with respective correlation values: {cpa_results_coefficients[:5]}")

print(f"Correct key byte: {correct_key[key_byte_index]}")

Top 5 key byte predictions: [125  78  83 236 247] with respective correlation values: [215.88852459  77.84987893  76.59183673  76.37506336  76.28249744]
Correct key byte: 125


In [24]:
# Full attack:
from tqdm.notebook import tqdm

key_predictions = np.zeros((16, 256))

for key_byte_index in tqdm(range(16), position=0, desc=f"Attacking key byte {key_byte_index}", leave=False, colour='green', ncols=80):

  for key_guess in tqdm(range(256), position=1, desc=f"Key guess: {hex(key_guess)}", leave=False, colour='red', ncols=80):

    hamming_weights = [hamming_weight(process_aes(byte, key_guess)) for byte in test_shard['pts'][key_byte_index]]

    # Pearson's correlation coefficient:
    # 
    #
    # r = cov(X,Y) / stddev(X)*stddev(Y)
    # 
    # cov(X,Y) = E[(X-X_)*(Y-Y_)]   => sum((X-))
    # stddev(X) = sqrt((X-X_)^2)

    hamming_weights_mean = np.mean(hamming_weights)
    hamming_weights_stddev = np.std(hamming_weights)
    hamming_weights_diff = (hamming_weights - hamming_weights_mean).reshape((256,1))
    
    covariance = np.sum(hamming_weights_diff*traces_diff, axis=0)
    pearson_correlation = covariance/((hamming_weights_stddev * traces_stddev).transpose())
    
    # Point in the trace where the correlation is biggest -> where the leak is happening
    key_predictions[key_byte_index][key_guess] = np.max(abs(pearson_correlation))

Attacking key byte 0:   0%|                              | 0/16 [00:00<?, ?it/s]

Key guess: 0xff:   0%|                                  | 0/256 [00:00<?, ?it/s]

Key guess: 0xff:   0%|                                  | 0/256 [00:00<?, ?it/s]

Key guess: 0xff:   0%|                                  | 0/256 [00:00<?, ?it/s]

Key guess: 0xff:   0%|                                  | 0/256 [00:00<?, ?it/s]

...

Key guess: 0xff:   0%|                                  | 0/256 [00:00<?, ?it/s]

Key guess: 0xff:   0%|                                  | 0/256 [00:00<?, ?it/s]

Key guess: 0xff:   0%|                                  | 0/256 [00:00<?, ?it/s]

Key guess: 0xff:   0%|                                  | 0/256 [00:00<?, ?it/s]

Key guess: 0xff:   0%|                                  | 0/256 [00:00<?, ?it/s]

In [25]:
def verify_CPA(correct_key, key_predictions):
  for i, k in enumerate(key_predictions):
    cpa_results_indices = np.argsort(k)[::-1]
    cpa_results_coefficients = np.asarray(k)[cpa_results_indices]

    if correct_key[i] == cpa_results_indices[0]:
      print(f"✔️ Key byte {cpa_results_indices[0]} guessed - pearson's coefficient = {cpa_results_coefficients[0]}")
    else:
      print(f"❌ Predicted byte {cpa_results_indices[0]}, but correct byte is {correct_key[i]}")

In [26]:
verify_CPA(correct_key, key_predictions)

✔️ Key byte 125 guessed - pearson's coefficient = 215.88852459016394
✔️ Key byte 3 guessed - pearson's coefficient = 216.66276150627615
✔️ Key byte 25 guessed - pearson's coefficient = 223.93584603047313
✔️ Key byte 126 guessed - pearson's coefficient = 220.75019952114926
✔️ Key byte 249 guessed - pearson's coefficient = 158.6983184965381
✔️ Key byte 41 guessed - pearson's coefficient = 186.16216216216216
✔️ Key byte 85 guessed - pearson's coefficient = 195.80511571254567
✔️ Key byte 69 guessed - pearson's coefficient = 188.58823529411765
✔️ Key byte 106 guessed - pearson's coefficient = 165.45864661654136
✔️ Key byte 159 guessed - pearson's coefficient = 179.50877192982455
✔️ Key byte 127 guessed - pearson's coefficient = 231.42380952380952
✔️ Key byte 133 guessed - pearson's coefficient = 179.42311770943797
✔️ Key byte 139 guessed - pearson's coefficient = 185.25345622119815
✔️ Key byte 202 guessed - pearson's coefficient = 178.7356173238526
✔️ Key byte 50 guessed - pearson's coeffic

In [40]:
def CPA_attack(traces, plaintext, num_traces, aes_key_size=16):
  
#   num_traces = traces.shape[0]
  key_predictions = np.zeros((aes_key_size, num_traces))
  
  traces_mean = np.mean(traces, axis=0)
  traces_stddev = np.std(traces, axis=0)

  traces_diff = traces - traces_mean.transpose()

  for key_byte_index in tqdm(range(16), position=0, desc=f"Attacking key byte", leave=False, colour='green', ncols=80):

    for key_guess in tqdm(range(num_traces), position=1, desc=f"Key guess", leave=False, colour='red', ncols=80):

      hamming_weights = [hamming_weight(process_aes(byte, key_guess)) for byte in plaintext[key_byte_index]]

      # Pearson's correlation coefficient:
      # 
      #
      # r = cov(X,Y) / stddev(X)*stddev(Y)
      # 
      # cov(X,Y) = E[(X-X_)*(Y-Y_)]   => sum((X-))
      # stddev(X) = sqrt((X-X_)^2)

      hamming_weights_mean = np.mean(hamming_weights)
      hamming_weights_stddev = np.std(hamming_weights)
      hamming_weights_diff = (hamming_weights - hamming_weights_mean).reshape((num_traces,1))
      
      covariance = np.sum(hamming_weights_diff*traces_diff, axis=0)
      pearson_correlation = covariance/((hamming_weights_stddev * traces_stddev).transpose())
      
      # Point in the trace where the correlation is biggest -> where the leak is happening
      key_predictions[key_byte_index][key_guess] = np.max(abs(pearson_correlation))
  
  return key_predictions

In [36]:
test_shard['traces'][:10, :,0]

array([[-0.375  , -0.11475, -0.2451 , ..., -0.2593 , -0.2163 , -0.05713],
       [-0.3462 , -0.1006 , -0.274  , ..., -0.2163 , -0.2886 ,  0.04395],
       [-0.187  , -0.2451 , -0.2886 , ..., -0.3174 ,  0.0879 , -0.1294 ],
       ...,
       [-0.2163 , -0.08594, -0.3174 , ..., -0.2163 , -0.187  , -0.1294 ],
       [-0.4907 , -0.1582 , -0.2451 , ..., -0.2451 , -0.1006 , -0.08594],
       [-0.2886 , -0.1729 , -0.2886 , ..., -0.274  , -0.0713 , -0.11475]],
      dtype=float16)

In [42]:
# Lower the number of traces and try again
num_traces = 50
cpa_input_traces = test_shard['traces'][:num_traces,:,0]
cpa_input_plaintexts = test_shard['pts'][:,:num_traces]

cpa_result = CPA_attack(cpa_input_traces, cpa_input_plaintexts, num_traces=num_traces)

verify_CPA(correct_key, cpa_result)

Attacking key byte:   0%|                                | 0/16 [00:00<?, ?it/s]

Key guess:   0%|                                         | 0/50 [00:00<?, ?it/s]

Key guess:   0%|                                         | 0/50 [00:00<?, ?it/s]

Key guess:   0%|                                         | 0/50 [00:00<?, ?it/s]

Key guess:   0%|                                         | 0/50 [00:00<?, ?it/s]

...

Key guess:   0%|                                         | 0/50 [00:00<?, ?it/s]

Key guess:   0%|                                         | 0/50 [00:00<?, ?it/s]

Key guess:   0%|                                         | 0/50 [00:00<?, ?it/s]

Key guess:   0%|                                         | 0/50 [00:00<?, ?it/s]

❌ Predicted byte 16, but correct byte is 125
✔️ Key byte 3 guessed - pearson's coefficient = 43.48257222739982
✔️ Key byte 25 guessed - pearson's coefficient = 42.09586324786326
❌ Predicted byte 6, but correct byte is 126
❌ Predicted byte 30, but correct byte is 249
✔️ Key byte 41 guessed - pearson's coefficient = 35.60594059405942
❌ Predicted byte 15, but correct byte is 85
❌ Predicted byte 2, but correct byte is 69
❌ Predicted byte 27, but correct byte is 106
❌ Predicted byte 32, but correct byte is 159
❌ Predicted byte 46, but correct byte is 127
❌ Predicted byte 13, but correct byte is 133
❌ Predicted byte 32, but correct byte is 139
❌ Predicted byte 24, but correct byte is 202
❌ Predicted byte 13, but correct byte is 50
❌ Predicted byte 9, but correct byte is 115


Less traces produce less accurate key prediction, which was expected.